# Reading FITS images from LB telescopes
Fully functional class. Requires access to images and a list of the folders where to look for them.


Basic packages: ``numpy``, ``pandas``, ``matplotlib``, ``astropy``.

In [8]:
import os, sys, glob
from pathlib import Path
import datetime as dt
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.patches as patches
%matplotlib qt

from astropy.io import fits
from astropy import units as u
from astropy.wcs import WCS
# from astropy.wcs.wcsapi import SlicedLowLevelWCS
from astropy.wcs.utils import skycoord_to_pixel
from astropy.visualization import SimpleNorm, simple_norm
from astropy.visualization import make_lupton_rgb
from astropy.visualization.wcsaxes import add_scalebar
from astropy.visualization.wcsaxes import SphericalCircle
from astropy.coordinates import Angle
from astropy.coordinates import SkyCoord
from astropy.coordinates import get_body, EarthLocation
from astropy.nddata import Cutout2D
from astropy.time import Time
from math import ceil




In [54]:
class image_viewer:
    def __init__(self, directory: str = '',
               list_available = False,
               folder_list = False,
               previous_df = False,
               print_error = True):
        """Class to quickly open FITS images. Searches in given directory.
        
        Attributes
        ---------
        directory : str
            Directory where images are stored. If none given look in current working directory.

        list_available : bool : False
            Wether to print the resulting dataframe of found images or not

        folder_list : optional, list of str
            Extra directories to inspect for images and save their folder path from working directory
        
        previous_df : optional, pd.DataFrame or str
            Previous df with files to be added to the new one of the files found in ``folder_list``

        print_error: bool, optional
            If False, no error warnings will be printed (good for reading large datasets)
        
        Methods
        --------
        return_index()
            Returns the image path and index in the datafile given one or the other.
        
        header_info()
            Method to view general header info.

        view()
            Method to view images.
        
        view_multiple()
            Method to view multiple images in subplots of a figure
        """
        self.folder_list = folder_list
        print('Current working directory: ' + os.getcwd())
        if directory=='':
            directory = os.getcwd()
        if directory != os.getcwd():
            self.dir_img = os.path.join(os.getcwd(),directory)
        else: self.dir_img = directory
        print('Image directory defined: ' + self.dir_img)

        # list of images in dir_img and where were they
        files = list(Path(self.dir_img).glob('*.fits'))
        folder_found = ['']*len(files)
        # list of images in the different folders of folder_list and the corresponding folder
        if folder_list!= False:
            for fl in folder_list:
                fi = list(Path(os.path.join(self.dir_img, fl)).glob('*.fits'))
                files=files+fi
                folder_found =folder_found+[fl]*len(fi)

        files_data = []
        # creation of data dictionary
        for k, f in enumerate(files):
            try:
                name = f.name
                path = str(f.resolve())
                try: telescope, camera, date_time, object, filter = name.split('_')
                except: 
                    if print_error: print('ERROR WITH FILENAME FORMAT CONVENTION EXPECTED')
                size_MB = f.stat().st_size / 1e6
                created = pd.to_datetime(f.stat().st_ctime, unit="s")
                files_data.append({"filename": name, "path": path, "telescope": telescope, 'camera': camera,
                                   "object": object, "filter": filter[:-5], "size_MB": size_MB,
                                   "date_time": pd.to_datetime(date_time, format='%Y-%m-%d-%H-%M-%S-%f'),
                                   "folder_found": folder_found[k]})
            except: 
                if print_error: print('Error with file: %s'%f)
                
        if len(files)==0:
            print('WARNING: NO IMAGE FILES FOUND')
            return
        # creation of dataframe
        df_files = pd.DataFrame(files_data).sort_values("filename").reset_index(drop=True)
        # Addition of previous dataframe
        if type(previous_df) != bool:
            if type(previous_df) != pd.DataFrame:
                if type(previous_df) == str:
                    if previous_df[-3:] == 'pkl': previous_df = pd.read_pickle(previous_df)
                    elif previous_df[-3:] == 'csv' : previous_df = pd.read_csv(previous_df)
                    else: 
                        print('ERROR: unrecognized DataFrame format. Use \'.pkl\' or \'.csv\'.')
                        return
            self.df_files = pd.concat([df_files, previous_df], ignore_index = True).drop_duplicates(subset = 'filename', keep= 'last')
        else: self.df_files = df_files
        # print available images if requested
        if list_available:
            print(self.df_files)
        print('Total number of images found: ', len(self.df_files))

        # Store gravitational lens objects
        grav_lens = ['QSO0957+561', 'Q2237+030', 'MG1654+1346', 'SDSSJ1004+4112', 'LBQS1333+0113', 'SDSSJ0819+5356',
             'EinsteinCross', 'DESI-350.3458-03.5082', 'ZTF25abnjznp']
        # EinsteinCross and Q2237+030 are the same object (?)
        grav_lens_ra = ['10 01 20.692 h', '22 40 30.234 h', '16 54 41.796 h', '10 04 34.936 h', '13:35:34.8 h', '08 19 59.764 h',
                        '22 40 30.271 h', '350.3458d', '07:16:34.5h']
        grav_lens_dec = ['+55 53 55.59 d', '+03 21 30.63 d', '+13 46 21.34 d', '+41 12 42.66 d', '+01 18 05.5 d', '+53 56 24.63 d',
                         '+03 21 31.03 d', '-03.5082d', '+38:21:08d']
        grav_data = []
        for i in range(len(grav_lens)):
            grav_data.append({
                'object' : grav_lens[i],
                'ra' : Angle(grav_lens_ra[i]),
                'dec' : Angle(grav_lens_dec[i])
                })
        self.df_grav_lens = pd.DataFrame(grav_data).sort_values('object').reset_index(drop=True)
    
    def return_index(self, image):
        """
        Returns the image path and index in the datafile given one or the other.

        Parameters
        ----------
        image: int / str
            int - image index in datafile \n
            str - image path
        """
        if type(image)==int:
            image_str = self.df_files.loc[image].filename
            image_int = image
        else: 
            image_str = image
            try: image_int = self.df_files.index[self.df_files['filename']==image].to_list()[0]
            except:
                print('\n ERROR: FILENAME NOT FOUND')
                return
        if self.folder_list != False:
            folder_name = self.df_files.iloc[image_int].folder_found
            image_str = os.path.join(folder_name, image_str)
        return image_str, image_int
    

    def image_finder(self, object, 
                     date = None, 
                     filter = None,
                     return_df = False,
                     printeo = False
                     ):
        """
        Method to identify the fits file that match an observation object, date and filter.
        
        Parameters
        ----------
        object : index / str
            Either the iloc or string to the object in self.df_grav_lens

        date : 'YYYY-MM-DD' (optional hh-mm-ss)
            If no date is supplied, return possible options

        filter : str
            Desired filter. If None, return possible options
        """
        try:
            if type(object) == str:
                obj_int = self.df_grav_lens.index[self.df_grav_lens['object'] == object].tolist()
                if obj_int == []:
                    print('ERROR: OBJECT NAME NOT REGISTERED.\n  Try with one of: ', self.df_grav_lens['object'].tolist())
                obj_str = object
            if type(object) == int:
                obj_str = self.df_grav_lens['object'].iloc[object]
                obj_int = object
        except: 
            print('ERROR: No previously known object was found.\n  Try with one of: ', self.df_grav_lens['object'].tolist())
        
        df_filtered = self.df_files[self.df_files["object"]==obj_str].copy()
        # df_filtered['day'] = df_filtered['date_time'].dt.date

        if date == None:
            print('Available date observations:')
            print(df_filtered.groupby(['object', 'folder_found']).size())
        
        if date != None:
            if type(date) == str:
                df_filtered = df_filtered[df_filtered['folder_found'] == date]
            if type(date) == list:
                df_filtered = df_filtered[df_filtered['folder_found'] in date]
            # df_filtered = df_filtered[df_filtered['day'] == dt.datetime.strptime(date, '%Y-%m-%d').date()]
            # print('Available observations:')
            # print(df_filtered.groupby(['object', 'filter']).size())

        if filter != None:
            df_filtered = df_filtered[df_filtered['filter'] == filter]
        
        if return_df == True:
            print('Matching index: ')
            print(df_filtered.index.tolist())
            return df_filtered
        else:
            return df_filtered.index.tolist()


    def header_info(self, image,
                    interesting_keys = ['INSTRUME', 'OBJECT', 'FILTER', 'INTEGT', 'DATE-OBS',
                                        'RA', 'DEC', 'NAXIS1', 'NAXIS2', 'SCALE', 'FOVX', 'FOVY',
                                        'CCW', 'CRPIX1', 'CRPIX2', 'FWHM']
                                        ):
        """Method to view general header info.
        
        Parameters
        ----------
        image : int / str
            int - index of desired file in dataframe \n
            string - path to desired fits file
            
        interesting_keys: list / 'all'
            list - list of strings with header keyword \n
            'all' - will print the whole header
        """
        image_str, image_int = self.return_index(image)
        
        # Extracting data from header
        with fits.open(os.path.join(self.dir_img, image_str)) as hdul:
            heads = hdul[0].header
            hdul.close()
        # printing basic header info
        print('Image: %s'%image_str)
        print('\n   --- HEADER DATA ---')
        if type(interesting_keys) == str and interesting_keys!='all':
                interesting_keys = [interesting_keys]
        try:
            if type(interesting_keys) == str and interesting_keys=='all':
                print(repr(heads))
            else:
                for k in interesting_keys:
                    if heads.comments[k]!='':
                        print(k, ' = ', heads[k], '  ---  ', heads.comments[k])
                    else:
                        print(k, ' = ', heads[k])
        except:
            print('WARNING: WRONG interesting_keys PARAMETER.')
            print('         Header parameter not recognized. Try the string \'all\' to view the full header')

    def view_image(self, image,
                    RGB = False,
                    nrows_ncols = None,
                    figsize = None,
                    manipulation_kw = {
                       'centered' : True,
                       'zoom' : False,
                       'stretch' : 'linear',
                       'percentile' : None,
                       'vminmax' : (None, None)
                       },
                    plotting_kw = {
                        'cmap' : 'gray',
                        'scalebar_arcsec' : 5,
                        'scalebar_frame' : False,
                        'add_circle' : None
                        },
                    RGB_kw = {
                        'stretch' : 5,
                        'Q' : 8,
                        'minimum' : None
                        },
                    RGB_norm_kw = {
                        'vmax' : None,
                        'max_percentile' : 99,
                        'max_sky' : False
                        }
                    ):
        """
        Method to view images. Takes dictionary keywords for ``data_manipulation`` and ``plotting``.
        """
        # Multiple images
        if type(image) == list and RGB == False:
            print('------\nViewing multiple images:')
            n_image = len(image)
            if nrows_ncols == None:
                if n_image <= 3: nrows_ncols = (1, n_image)
                else: nrows_ncols = (ceil(np.sqrt(n_image)), ceil(np.sqrt(n_image)))
            image_list = image

        # Simple image Non RGB
        if type(image) != list:
            print('------\nViewing image:')
            n_image, nrows_ncols = 1, (1,1)
            image_list = [image]
        # RGB image
        if RGB == True: 
            n_image = 1
            colors = ['R', 'G', 'B']
            cutout_RGB = []
            # if manipulation_kw['stretch']!='linear':
            #     print('Overriding for a linear stretch')
            #     manipulation_kw['stretch']='linear'
            print('------\nRGB color composite image:')
            if n_image == 1: nrows_ncols = (1,1)
            image_list = image

        self.nr_nc = nrows_ncols
        n_data = len(image_list)

        # if manipulation and plotting are dicts, use the same setup for all images
        if type(manipulation_kw) == dict: manipulation_kw = [manipulation_kw]*n_data
        if type(plotting_kw) == dict: plotting_kw = [plotting_kw]*n_data

        fig, axes = plt.subplots(self.nr_nc[0], self.nr_nc[1],
                                 figsize = figsize)#, layout = 'constrained')
        if n_image == 1: axes = [axes]
        axes = np.array(axes).reshape(-1)
        
        for i, (img, m_k, p_k) in enumerate(zip(image_list, manipulation_kw, plotting_kw)):
            self.img_str, self.img_int = self.return_index(img)
            cutout, norm = self.data_manipulation(self.img_str, **m_k)

            if RGB == False:
                print('    Object: ',self.df_files.object.loc[self.img_int],
                      '  -  Filter: ',self.df_files['filter'].loc[self.img_int])
                self.plotting(cutout, norm, fig, axes[i], i,
                              **p_k)
            else:
                # Extracting data from header
                with fits.open(os.path.join(self.dir_img, self.img_str)) as hdul:
                    heads = hdul[0].header
                    hdul.close()
                if i==0: print('    Object: ', self.df_files.loc[self.img_int].object)
                print('    - ',colors[i],': ', self.df_files['filter'].loc[self.img_int])
                # min and max for manual norm, if max_sky is set, use it to obtain max as max_sky * sky_flux
                vmin = heads['FLUXSKY']
                if 'vmax' in RGB_norm_kw.keys(): vmax = RGB_norm_kw['vmax']
                if RGB_norm_kw['max_sky'] != False: vmax = RGB_norm_kw['max_sky']*heads['FLUXSKY']
                if vmax == None: vmax = np.max(cutout.data)
                # manual normalization
                data = (cutout.data - vmin)/(vmax-vmin)
                data_mask = data < 1e-3
                data[data_mask] = 1e-3
                cutout_RGB.append(data)
                # norm_RGB = simple_norm(cutout.data, stretch = 'linear', 
                #                        vmin = vmin, max_percent = RGB_norm_kw['max_percentile'],
                #                        vmax = RGB_norm_kw['vmax'],
                #                        clip = True)
                # cutout_RGB.append(norm_RGB(cutout.data))
                # print(colors[i], ' data:')
                # print(vmin, type(vmin))
                # print('  - Original sky flux', heads['FLUXSKY'])
                # print('- Original min and max values: ', np.min(cutout.data), np.max(cutout.data))
                # print('- After normalization: ', np.min(cutout_RGB[i]), np.max(cutout_RGB[i]))
                if i == len(image_list)-1:
                    # for k in range(3):
                        
                    rgb_default = make_lupton_rgb(cutout_RGB[0].data, cutout_RGB[1].data, cutout_RGB[2].data,
                                                  **RGB_kw)
                    self.plotting(cutout, norm, fig, axes[0],0,
                                  RGB = True, rgb_data = rgb_default,
                                  **plotting_kw[i])

                    # axes[0].remove()
                    # POSSIBLE IMPROVEMENT: USE EACH IMAGE'S WCS TO ALIGN PIXELS
                    # ax = fig.add_subplot(1,3, 1, projection = cutout.wcs)
                    # ax.imshow(rgb_default, origin='lower')
                    # ax.set_title('Lupton RGB')
                    # ax = fig.add_subplot(1,3, 2, projection = cutout.wcs)
                    # ax.imshow(cutout_RGB[-1].data, origin = 'lower')
                    # ax.set_title('R filter normalized data')
                    # ax = fig.add_subplot(1,3, 3, projection = cutout.wcs)
                    # ax.imshow(cutout.data, origin = 'lower')
                    # ax.set_title('Non normalized data')
       
        # if RGB:
        #     print('    Object: ', self.df_files.iloc[self.img_int].object)
        #     for i, (img, m_k, p_k) in enumerate(zip(image_list, manipulation_kw, plotting_kw)):
        #         print('    - ',colors[i],': ', self.df_files['filter'].iloc[self.img_int])
        #         self.img_str, self.img_int = self.return_index(img)

        #     fig, ax = plt.subplots(subplot_kw={'projection': wcs})
        plt.tight_layout()
        plt.show()

    def data_manipulation(self, image_str,
                          centered = True, 
                          zoom = False,
                          stretch = 'linear',
                          percentile = None,
                          vminmax = (None, None)
                          ):
        """
        Method to prepare images for manipulation. It is internally called. Crops the image and sets visualization normalization and stretch.

        Parameters
        ---------
        image : int / string / list
            int - index of desired file in dataframe \n
            string - path to desired fits file \n

        centered : True or tuple, optional
            (x,y) - int for pix coordinates \n
            (RA, DEC) - wcs coordinates. Accepting both strings or angle values

        zoom : False or Value or Tuple, optional
            int / (int, int) - pixel size in x and y axis \n
            Angle / (Angle, Angle) - angular size in RA and DEC
        
        stretch : str, optional
            Image stretch to enhance detail visualization \n
            ``linear``, ``sqrt``, ``power``, ``log``, ``sinh``, ``asinh``
        
        percentile : int or tuple, optional
            ``int`` - Middle percentile of values to consider for normalization; 
            ``tuple`` - Lower and upper percentile of values to consider for normalization
        
        vminmax : tuple, optional
            Min and max pixel values for normalization. Overrides ``percentile``.
            If set as None, keeps the absolute min or max of image
        """
        

        # Extracting data from header
        with fits.open(os.path.join(self.dir_img, image_str)) as hdul:
            data = hdul[0].data.astype(np.float32)
            heads = hdul[0].header
            wcs = WCS(heads)
            hdul.close()
        
        # obtaining central px coordinates
        x_shape = data.shape[1]
        y_shape = data.shape[0]
        if centered == True:
            center_px = (x_shape//2, y_shape//2)
        if type(centered)==tuple:
            if type(centered[0]) == int: # input in px units
                center_px = tuple(centered)
            elif type(centered[0]) == str: # input in str to be converted to deg
                center_angle = SkyCoord(centered[0], centered[1], frame = 'icrs')
                center_px = skycoord_to_pixel(center_angle, wcs, origin=0)
            else:
                center_angle = SkyCoord(centered[0], centered[1], frame = 'icrs')
                center_px = skycoord_to_pixel(center_angle, wcs, origin=0)
        
        # setting zoom
        if zoom == False:
            zoom = (x_shape, y_shape)
        if type(zoom) == str:
            zoom = Angle(zoom)
        if type(zoom)== tuple:
            if type(zoom[0]) == str:
                zoom = (Angle(zoom[0]), Angle(zoom[1]))
        if type(zoom)==tuple:
            zoom = zoom[::-1]
        
        # slicing image
        try:
            cutout = Cutout2D(data, position = center_px, size = zoom, wcs = wcs)
        except:
            print('\n --- \nERROR: the cutout region is outside of the image.')
            return

        # norm definition
        if type(percentile) == int or percentile == None:
            percentile_minmax = (None, None)
        if type(percentile) == tuple:
            percentile_minmax = percentile
            percentile = None
        if stretch not in {'linear', 'sqrt', 'power', 'log', 'sinh', 'asinh'}:
            print('ERROR: Stretch should be one of \'linear\', \'sqrt\', \'power\', \'log\', \'sinh\', \'asinh\'')
            plt.close()
            return
        norm = simple_norm(cutout.data, stretch = stretch, 
                           vmin = vminmax[0], vmax = vminmax[1],
                           percent = percentile,
                           min_percent = percentile_minmax[0],
                           max_percent = percentile_minmax[1])
        
        return cutout, norm
        
    def plotting(self,
                 cutout, norm, fig, ax, ax_i,
                cmap = 'gray',
                scalebar_arcsec = 5, scalebar_frame = False,
                add_circle = None,
                RGB = False,
                rgb_data = False
                ):
        """
        Method to plot images, obtains edited data from ``self.data_manipulation()``.

        Parameters
        ---------
        cutout : Cutout2D
            Selected cutout object from ``data_manipulation``

        norm : Norm
            Selected norm from ``data_manipulation``

        cmap : str, optional
            Select the desired colormap for the image

        scalebar_arcsec : int, optional
            Angular size of scalebar in arcsec units
        
        scalebar_frame : bool, optional
            Add frame or not

        add_circle : dict, list of dicts or None, optional
            Parameters to plot a circle overlay. If None, no circle is plotted. If multiple circles are desired, enter a list of dicts.\n
            Expected keys: \n
                'center' : tuple 
                    (RA, DEC) coordinates as astropy Angle or SkyCoord
                'size' : astropy.units.Quantity
                    Angular size (e.g., astropy Angle with units).
                'color' : str, optional
                    Circle edge color.
                'label' : str, optional
                    Label for the circle to use in legend.
            
        fig_kwrds : None or dict, optional
            Dict with all the keywords desired to insert in ``plt.subplots()``

        figure : None or dict ..... tuple or axis
            Dict used by view_multiple method. Expected keys: \n
                'is_simple' : bool
                'create_fig' : bool
                    True or False
                'figsize' : tuple
                    Looked at if ``create_fig = True``
                'nrows_ncols' : tuple
                    Looked at if ``create_fig = True``
                'fig' : plt.figure object
                    Looked at if ``create_fig = False``
                'ax' : plt.axis object
                    Looked at if ``create_fig = False``
                'im_i' : int
                    Subplot index (image index). Looked at if ``create_fig = False``

            None - creates normal figure, does not return nothing \n
            tuple (int, int) - creates figure with specified conditions. Returns (fig, ax) \n
            tuple (ax, int, int) - plots image in specified ax[int,int]
        """
        # # WCS projection (for simple figures)
        # if figure['is_simple'] == True:
        #     fig, ax = plt.subplots(subplot_kw=dict(projection=cutout.wcs), **fig_kwrds)
        # # Figure creation if multiple figures are in use
        # if figure['is_simple'] == False:
        #     create_fig = figure['create_fig']
        #     nrows_ncols = figure['nrows_ncols']
        #     im_i = figure['im_i']
        # if create_fig == True:
        #     figsize = figure['figsize']
        #     fig, axes = plt.subplots(nrows_ncols[0], nrows_ncols[1],
        #                             #  projection = None,
        #                                 figsize = figsize)
        #     axes = np.array(axes).reshape(-1)
        #     ax = axes[0]
        # else:
        #     fig = figure['fig']
        #     ax = figure['ax']
        with fits.open(os.path.join(self.dir_img, self.img_str)) as hdul:
            heads = hdul[0].header
            hdul.close()
        ax.remove()
        ax = fig.add_subplot(self.nr_nc[0], self.nr_nc[1], ax_i+1, projection = cutout.wcs)
        if RGB == False:
        # colorbar
            cax = ax.imshow(cutout.data,
                            norm = norm, origin = 'lower',
                            cmap = cmap)
            cbar = plt.colorbar(cax)
            cbar.set_label('ADU', rotation=270, labelpad=15)
            cbar.ax.tick_params(labelsize=10)
        else:
            ax.imshow(rgb_data, origin = 'lower')

        # Scale bar choosing color depending on luminance of cmap
        scalebar_angle = scalebar_arcsec/3600*u.deg
        rgba = plt.get_cmap(cmap)(0.0)
        luminance = 0.299*rgba[0] + 0.587*rgba[1] + 0.114*rgba[2]
        scalebar_color = 'white' if (luminance < 0.5 and scalebar_frame == False) else 'black'
        add_scalebar(ax, scalebar_angle, label="%s arcsec"%str(scalebar_arcsec), color=scalebar_color, frame=scalebar_frame)
        # Axis and title
        ax.set(xlabel='RA', ylabel='DEC')
        ax.coords.grid(color='gray', alpha=0.5, linestyle='solid')
        title_str = (r'$\bf{Object}$: %s - $\bf{Telescope}$: %s - $\bf{Seeing}$: %.1f$^{\prime\prime}$''\n'
                    r'$\bf{Camera}$: %s - $\bf{Filter}$: %s - $\bf{Integration}$: %s s''\n'
                    r'$\bf{SNR}$: %s - $\bf{Date time}$: %s - $\bf{Moon D}$: %.1fº'
                    %(self.df_files.iloc[self.img_int]['object'],
                    self.df_files.iloc[self.img_int]['telescope'],
                    (float(heads['FWHM'])*float(heads['SCALE'])),
                    self.df_files.iloc[self.img_int]['camera'],
                    self.df_files.iloc[self.img_int]['filter'],
                    heads['INTEGT'], heads['OBJECSNR'],
                    self.df_files.iloc[self.img_int]['date_time'].strftime("%Y-%m-%d %H:%M"),
                    self.get_moon_distance(self.img_int).deg))
        ax.set_title(title_str)
        ax.minorticks_on()

        # Optional plot of circles
        if add_circle is not None:
            if type(add_circle) != list:
                add_circle = [add_circle]
            for d_circle in add_circle:
                center = d_circle.get('center')
                size = d_circle.get('size')
                color = d_circle.get('color')
                label = d_circle.get('label')
                c = SphericalCircle((Angle(center[0]), Angle(center[1])),
                                    Angle(size),
                                    edgecolor = color,
                                    facecolor = 'none',
                                    transform = ax.get_transform('icrs'))
                ax.add_patch(c)
 

    def read_data(self, image, header = False):
        """Method to view images."""
        image_str, image_int = self.return_index(image)
        print('Reading ', image_str)

        # Extracting data from header
        with fits.open(os.path.join(self.dir_img, image_str)) as hdul:
            data = hdul[0].data.astype(np.float32)
            head = hdul[0].header
            hdul.close()

        if header == False: return data
        else: return head
    
    def get_moon_distance(self, image):
        """
        Method to calculate the angular separation with the Moon in degrees for a given observation.
        
        Parameters
        ---------
        image : int or str
            index of observation image or string to .fits file.
            
        Returns:
            astropy.Angle object with angular separation"""

        image_str, image_int = self.return_index(image)
        with fits.open(os.path.join(self.dir_img, image_str)) as hdul:
            heads = hdul[0].header
            hdul.close()
        RA = str(heads['RA']) + ' d'
        DEC = str(heads['DEC']) + ' d'
        time = Time(self.df_files.iloc[image_int]['date_time'])
        loc = EarthLocation.of_site('Observatorio del Teide')
        moon_coords = get_body('moon', time = time, location = loc)
        moon_coords = SkyCoord(ra = moon_coords.ra, dec = moon_coords.dec, frame = 'icrs', unit = u.deg)
        obj_coords = SkyCoord(ra = Angle(RA), dec = Angle(DEC), frame = 'icrs', unit = u.deg)
        sep = obj_coords.separation(moon_coords)
        # print('angle %.1fº'%sep.deg)
        return sep
        # print('Time of observation: %s'%time)
        # print('Object coordinates: \n - RA: %s\n - DEC: %s' %(obj_coords.ra, obj_coords.dec))
        # print('Moon coordinates: \n - RA: %s\n - DEC: %s' %(moon_coords.ra, moon_coords.dec))
        # print('Angular separation to Moon: ', sep)



In [70]:
folder = '2025-11-10'
filter = None
object_name = 'ZTF25abnjznp'
iv = image_viewer(directory = 'test_images', folder_list = [folder])
obj_int = iv.df_grav_lens.index[iv.df_grav_lens['object'] == object_name]
obj_coords = (iv.df_grav_lens['ra'].loc[obj_int],
              iv.df_grav_lens['dec'].loc[obj_int])

index = iv.image_finder(object_name, date=folder, filter = filter,
                        printeo = True, return_df = False)

iv.view_image([2,1,0], RGB = True,
              RGB_kw={'stretch' : 1,
                      'Q' : 8},
              RGB_norm_kw={'max_sky' : 1.1},
              manipulation_kw={'centered' : obj_coords,
                               'zoom' : '0 3 1 d'},
              plotting_kw={'add_circle' : {'center' : obj_coords,
                                           'size' : '0 0 10d',
                                           'color' : 'gray'}})

Current working directory: /Users/oscar/LB/grav_lens
Image directory defined: /Users/oscar/LB/grav_lens/test_images
Total number of images found:  4
------
RGB color composite image:
    Object:  ZTF25abnjznp
    -  R :  SDSSi
    -  G :  SDSSr
    -  B :  SDSSg


In [67]:
plt.savefig('RGB_test_bad.png', dpi=200)

In [6]:
folder = '2025-11-11'
filter = 'SDSSr'
filter = None

object_name = 'ZTF25abnjznp'
# object_name = 'DESI-350.3458-03.5082'
# object_name = 'EinsteinCross'
iv = image_viewer(directory = 'test_images', folder_list = [folder])#, previous_df='datafiles/df_2025-11-09.pkl')

obj_int = iv.df_grav_lens.index[iv.df_grav_lens['object'] == object_name]
obj_coords = (iv.df_grav_lens['ra'].loc[obj_int].values[0],
              iv.df_grav_lens['dec'].loc[obj_int].values[0])

index = iv.image_finder(object_name, date=folder, filter = filter)#[0]
print(index, index[::-1][1:])
# iv.view_image(index[::-1][1:], RGB = True,
iv.view_image(index, RGB = False,
                      figsize = (16,8),
                      manipulation_kw = {
                         'centered' : obj_coords,
                        #  'zoom' : '0 0 20 d',
                         'stretch' : 'linear'
                         },
                     RGB_kw= {'stretch' : 0.5,
                       'Q' : 8,
                       'minimum' : None
                       }
                     # RGB_norm_kw = {
                     #          'max_percentile' : 99.9,
                     #          'vmax' : None
                     #          })
                        #  {
                        #  'centered' : obj_coords,
                        #  'zoom' : '0 0 20 d',
                        #  'stretch' : 'sinh'
                        #  }]
                        )
# plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.07, wspace=0.3, hspace=0.4)


Current working directory: /Users/oscar/LB/grav_lens
Image directory defined: /Users/oscar/LB/grav_lens/test_images
Total number of images found:  4
[0, 1, 2, 3] [2, 1, 0]
------
Viewing multiple images:
    Object:  ZTF25abnjznp   -  Filter:  SDSSg
    Object:  ZTF25abnjznp   -  Filter:  SDSSr
    Object:  ZTF25abnjznp   -  Filter:  SDSSi
    Object:  ZTF25abnjznp   -  Filter:  SDSSzs


In [436]:
plt.savefig('result_images/'+object_name+'/'+object_name+'_linear_sinh_comp.png', dpi = 200)


In [442]:
folders = glob.glob('test_images/2025*')
folders = [os.path.split(f)[1] for f in folders]

iv = image_viewer(directory = 'test_images', folder_list = folders)#, previous_df='datafiles/df_2025-11-09.pkl')

object_name = 'DESI-350.3458-03.5082'
object_name = 'ZTF25abnjznp'
# object_name = 'EinsteinCross'
obj_int = iv.df_grav_lens.index[iv.df_grav_lens['object'] == object_name]
obj_coords = (iv.df_grav_lens['ra'].loc[obj_int].values[0],
              iv.df_grav_lens['dec'].loc[obj_int].values[0])

for fold in folders:
    indexes = iv.image_finder(object_name, date=fold[-10:])
    if indexes != []:
        iv.header_info(indexes[0], interesting_keys=['AIRMASS', 'ALT'])
    
    #     iv.view_image(indexes, RGB = False,
    #                   figsize = (16,8),
    #                   manipulation_kw = {
    #                      'centered' : obj_coords,
    #                      'zoom' : '0 0 20 d',
    #                      'stretch' : 'linear'
    #                      })
    #     plt.subplots_adjust(left=0.0, right=0.9, top=0.9, bottom=0.07, wspace=0.0, hspace=0.4)
    #     plt.savefig('result_images/'+object_name+'/'+object_name+'_'+fold[-10:]+'.png', dpi = 200)
    



Current working directory: /Users/oscar/LB/grav_lens
Image directory defined: /Users/oscar/LB/grav_lens/test_images
Total number of images found:  32
object        filter
ZTF25abnjznp  SDSSg     1
              SDSSi     1
              SDSSr     1
              SDSSzs    1
dtype: int64
Image: 2025-11-05/TTT3_iKon936-1_2025-11-06-02-33-01-072756_ZTF25abnjznp_SDSSg.fits

   --- HEADER DATA ---
AIRMASS  =  1.242002   ---   Airmass
ALT  =  53.624912   ---   [deg] Altitude
Series([], dtype: int64)
object        filter
ZTF25abnjznp  SDSSg     1
              SDSSi     1
              SDSSr     1
              SDSSzs    1
dtype: int64
Image: 2025-11-08/TTT3_iKon936-1_2025-11-09-02-05-33-447076_ZTF25abnjznp_SDSSg.fits

   --- HEADER DATA ---
AIRMASS  =  1.295143   ---   Airmass
ALT  =  50.544226   ---   [deg] Altitude
object        filter
ZTF25abnjznp  SDSSg     1
              SDSSi     1
              SDSSr     1
              SDSSzs    1
dtype: int64
Image: 2025-11-06/TTT3_iKon936-1_2025-1

In [441]:
iv.header_info(0,interesting_keys='all')

Image: 2025-11-02/TTT3_iKon936-1_2025-11-02-19-51-23-005585_EinsteinCross_SDSSg.fits

   --- HEADER DATA ---
SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                  -32 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                 2048                                                  
NAXIS2  =                 2048                                                  
IMAGETYP= 'SCIENCE '           / Type of image                                  
COMMENT ***************************                                             
COMMENT          TELESCOPE                                                      
COMMENT ***************************                                             
TELESCOP= 'TTT3    '           / Telescope name                                 
SITENAME= 'Teide Observatory (IAC)' / Telescope site name                       


In [182]:
iv = image_viewer(directory = 'test_images', folder_list = ['2025-11-08'])
# for i in range(4):
#     iv.header_info(i, interesting_keys='FLUXSKY')# iv.view_image(0)
iv.df_files
# iv.header_info(0, interesting_keys='all')

Current working directory: /Users/oscar/LB/grav_lens
Image directory defined: /Users/oscar/LB/grav_lens/test_images
Total number of images found:  4


,filename,path,telescope,camera,object,filter,size_MB,date_time,folder_found
0,TTT3_iKon936-1_2025-11-09-02-05-33-447076_ZTF2...,/Users/oscar/LB/grav_lens/test_images/2025-11-...,TTT3,iKon936-1,ZTF25abnjznp,SDSSg,16.79616,2025-11-09 02:05:33.447076,2025-11-08
1,TTT3_iKon936-1_2025-11-09-02-10-44-586606_ZTF2...,/Users/oscar/LB/grav_lens/test_images/2025-11-...,TTT3,iKon936-1,ZTF25abnjznp,SDSSr,16.79616,2025-11-09 02:10:44.586606,2025-11-08
2,TTT3_iKon936-1_2025-11-09-02-15-55-878040_ZTF2...,/Users/oscar/LB/grav_lens/test_images/2025-11-...,TTT3,iKon936-1,ZTF25abnjznp,SDSSi,16.79616,2025-11-09 02:15:55.878040,2025-11-08
3,TTT3_iKon936-1_2025-11-09-02-21-07-187714_ZTF2...,/Users/oscar/LB/grav_lens/test_images/2025-11-...,TTT3,iKon936-1,ZTF25abnjznp,SDSSzs,16.79616,2025-11-09 02:21:07.187714,2025-11-08


In [254]:
iv = image_viewer(directory = 'test_images', folder_list = ['2025-11-07'])
obj_coords = ('07:16:34.5h','+38:21:08d')
iv.view_image([2,1,0], RGB = True, 
              figsize = (12,8),
              manipulation_kw={
                    #  'centered' : obj_coords,
                     'zoom' : ('0 0 25 d', '0:0:25 d'),
                     'stretch' : 'linear'}
                     ,
              #        'percentile' : (10,100)},
                            
              plotting_kw = {
                #   'zoom' : ('0 0 50 d', '0:0:50 d')
              },
              RGB_kw= {'stretch' : 1,
                       'Q' : 1,
                       'minimum' : None
                       },
              RGB_norm_kw = {
                       'max_percentile' : 99.9,
                       'vmax' : None
                       })

Current working directory: /Users/oscar/LB/grav_lens
Image directory defined: /Users/oscar/LB/grav_lens/test_images
Total number of images found:  8
------
RGB color composite image:
    Object:  DESI-350.3458-03.5082
    -  R :  SDSSi
    -  G :  SDSSr
    -  B :  SDSSg
SEEING IN TITLE


In [28]:
iv.view_image(0, figsize = (12,8))
image_str, image_int = iv.return_index(0)
        
# Extracting data from header
with fits.open(os.path.join(iv.dir_img, image_str)) as hdul:
    data = hdul[0].data.astype(np.float32)
    hdul.close()
imin, imax = data.argmin(), data.argmax()
argmin = np.unravel_index(imin, data.shape)
argmax = np.unravel_index(imax, data.shape)
print(argmin, argmax)
data[argmax]

------
Viewing image:
    Object:  DESI-350.3458-03.5082   -  Filter:  SDSSg
(np.int64(0), np.int64(2047)) (np.int64(233), np.int64(1127))


np.float32(65211.562)

/Users/oscar/LB/grav_lens/.venv/lib/python3.13/site-packages/ipykernel/eventloops.py:145: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all Axes decorations.
  el.exec() if hasattr(el, "exec") else el.exec_()


In [257]:
iv = image_viewer(directory = 'test_images', folder_list = ['2025-11-08'])
# iv.df_files
obj_coords = ('07:16:34.5h','+38:21:08d')
indexes = [0,1,2,3]
# indexes = [4,5,6,7]
iv.view_image(indexes, 
                 figsize=(12,8),
                 manipulation_kw={
                     'centered' : obj_coords,
                     'zoom' : ('0 0 20 d', '0:0:20 d'),
                     'stretch' : 'linear',
                     'percentile' : (10,100)}
                            )
# centered' : obj_coords,
#                      'zoom' : ('0 0 25 d', '0:0:25 d'),
#                      'stretch' : 'sinh',
#                      'percentile' : (10,100),
#                      'scalebar_frame' : True,
#                      'add_circle' : {'center' : obj_coords,
#                                      'size' : '0:0:5d',
#                                      'color' : 'white'}

                            # 'percentile' : (50,90)})
# iv.view_color([3,1,0])

Current working directory: /Users/oscar/LB/grav_lens
Image directory defined: /Users/oscar/LB/grav_lens/test_images
Total number of images found:  4
------
Viewing multiple images:
    Object:  ZTF25abnjznp   -  Filter:  SDSSg
SEEING IN TITLE
    Object:  ZTF25abnjznp   -  Filter:  SDSSr
SEEING IN TITLE
    Object:  ZTF25abnjznp   -  Filter:  SDSSi
SEEING IN TITLE
    Object:  ZTF25abnjznp   -  Filter:  SDSSzs
SEEING IN TITLE


In [172]:
plt.savefig('ZTF_11_08.png', dpi=200)

In [ ]:
iv = image_viewer(directory = 'test_images', folder_list = ['2025-11-06'])#, previous_df='df_files.pkl')
# iv.df_files#.groupby(['telescope', 'folder_found']).size()

obj_coords = ('07:16:34.5h','+38:21:08d')
indexes = [0,1,2,3]

iv.view_multiple(indexes,
                 figsize = (12,8),
                 view_kwrds={
                     'centered' : obj_coords,
                     'zoom' : ('0 0 50 d', '0:0:20 d'),
                     'stretch' : 'linear',
                     'percentile' : (10,100),
                     'scalebar_frame' : True
                    #  'add_circle' : {'center' : obj_coords,
                    #                  'size' : '0:0:5d',
                    #                  'color' : 'white'}
                        })

Current working directory: /Users/oscar/LB/grav_lens
Image directory defined: /Users/oscar/LB/grav_lens/test_images
Total number of images found:  4


AttributeError: 'image_viewer' object has no attribute 'view_multiple'

In [63]:
obj_coords = ('350.3458d','-03.5082d')
indexes = [2,3,4,5]
obj_coords = ('07:16:34.5h','+38:21:08d')
indexes = [0,1,2,3]

iv.view_multiple(indexes,
                 figsize = (12,8),
                 view_kwrds={
                     'centered' : obj_coords,
                     'zoom' : ('0 0 25 d', '0:0:25 d'),
                     'stretch' : 'sinh',
                     'percentile' : (10,100),
                     'scalebar_frame' : True,
                     'add_circle' : {'center' : obj_coords,
                                     'size' : '0:0:5d',
                                     'color' : 'white'}
                        })

Viewing  2025-11-06/TTT3_iKon936-1_2025-11-07-02-19-33-541318_ZTF25abnjznp_SDSSg.fits
Viewing  2025-11-06/TTT3_iKon936-1_2025-11-07-02-24-44-679453_ZTF25abnjznp_SDSSr.fits
Viewing  2025-11-06/TTT3_iKon936-1_2025-11-07-02-29-56-000503_ZTF25abnjznp_SDSSi.fits
Viewing  2025-11-06/TTT3_iKon936-1_2025-11-07-02-35-07-118053_ZTF25abnjznp_SDSSzs.fits


In [244]:
iv = image_viewer('test_images', list_available=False, folder_list=['2025-11-05'])
# iv.df_files
# iv.header_info(2, interesting_keys='all')
# iv.view(4) #span_x=1000, span_y=1000)
iv.view(2, 
        # centered = ('350.3458d','-03.5082d'),#True,#('22h 40m 30.3s', '+3° 21′ 31″'),
        zoom = ('0 0 40 d', '0:0:40 d'),
        stretch = 'linear',
        # percentile = (1,99),
        # vminmax = (2e4, None),
        cmap = 'gray',
        scalebar_arcsec= 5,
        add_circle= {'center' : ('350.3458d','-03.5082d'),
                     'size' : '0:0:3d',
                     'color' : 'red'})
plt.tight_layout()

Current working directory: /Users/oscar/LB/grav_lens
Image directory defined: /Users/oscar/LB/grav_lens/test_images
Total number of images found:  10
Viewing  2025-11-05/TTT3_iKon936-1_2025-11-05-21-45-13-833208_DESI-350.3458-03.5082_SDSSg.fits


In [54]:
iv = image_viewer('test_images/2025-11-02')

for im_i, filter in zip([1,2],['g','r']):
    hdulist = fits.open(os.path.join(iv.dir_img,iv.return_index(im_i)[0]))
    hdu = hdulist[0]
    globals()[filter] = hdu.data
    hdulist.close()
def normalize(data):
    data_min, data_max = np.percentile(data, (50, 99))
    data = np.clip(data, data_min, data_max)
    return (data - data_min) / (data_max - data_min)
g_n = normalize(g)
r_n = normalize(r)
i = np.zeros_like(g)
rgb_default = make_lupton_rgb(g_n, r_n, i, Q=10, stretch=0.5)#, filename="ngc6976-default.jpeg")
fig, ax = plt.subplots()
ax.imshow(rgb_default, origin='lower')

Current working directory: /Users/oscar/LB/grav_lens
Image directory defined: /Users/oscar/LB/grav_lens/test_images/2025-11-02
Total number of images found:  4


In [41]:
from astropy.utils.data import get_pkg_data_filename

# Read in the three images downloaded from here:
g_name = get_pkg_data_filename('visualization/reprojected_sdss_g.fits.bz2')
r_name = get_pkg_data_filename('visualization/reprojected_sdss_r.fits.bz2')
i_name = get_pkg_data_filename('visualization/reprojected_sdss_i.fits.bz2')
g = fits.getdata(g_name)
r = fits.getdata(r_name)
i = fits.getdata(i_name)

rgb_default = make_lupton_rgb(i, r, g)
fig, ax = plt.subplots()
ax.imshow(rgb_default, origin='lower')

# Work with historic dataframe imported from .pkl file

In [149]:
df = pd.read_pickle('datafiles/df_2025-11-09.pkl')
#df.set_index('date_time').groupby('object').resample('D').size().unstack(0)
df['day'] = df['date_time'].dt.date
df

,filename,path,telescope,camera,object,filter,size_MB,date_time,folder_found,day
1332,TST_QHY411-3_2025-11-06-19-33-37-170516_C2025R...,/home/oscarsoler/work/red/2025-11-06/TST_QHY41...,TST,QHY411-3,C2025R2,SDSSg,604.93824,2025-11-06 19:33:37.170516,2025-11-06,2025-11-06
1353,TST_QHY411-3_2025-11-06-19-38-18-308631_C2025R...,/home/oscarsoler/work/red/2025-11-06/TST_QHY41...,TST,QHY411-3,C2025R2,SDSSr,604.93824,2025-11-06 19:38:18.308631,2025-11-06,2025-11-06
1443,TST_QHY411-3_2025-11-06-19-58-27-297772_C2025R...,/home/oscarsoler/work/red/2025-11-06/TST_QHY41...,TST,QHY411-3,C2025R2,Lum,604.93824,2025-11-06 19:58:27.297772,2025-11-06,2025-11-06
1451,TST_QHY411-3_2025-11-06-20-00-14-769813_C2025R...,/home/oscarsoler/work/red/2025-11-06/TST_QHY41...,TST,QHY411-3,C2025R2,Lum,604.93824,2025-11-06 20:00:14.769813,2025-11-06,2025-11-06
1469,TST_QHY411-3_2025-11-06-20-04-15-836393_C2025R...,/home/oscarsoler/work/red/2025-11-06/TST_QHY41...,TST,QHY411-3,C2025R2,SDSSr,604.93824,2025-11-06 20:04:15.836393,2025-11-06,2025-11-06
...,...,...,...,...,...,...,...,...,...,...
1047348,TTT3_iKon936-1_2025-11-07-01-18-54-611004_TOI-...,/home/oscarsoler/work/red/2025-11-06/TTT3_iKon...,TTT3,iKon936-1,TOI-7522,iz,16.79616,2025-11-07 01:18:54.611004,2025-11-06,2025-11-07
1047349,TTT3_iKon936-1_2025-11-07-01-22-00-835048_PSJ0...,/home/oscarsoler/work/red/2025-11-06/TTT3_iKon...,TTT3,iKon936-1,PSJ0147+4630,SDSSg,16.79616,2025-11-07 01:22:00.835048,2025-11-06,2025-11-07
1047350,TTT3_iKon936-1_2025-11-07-01-24-05-673433_PSJ0...,/home/oscarsoler/work/red/2025-11-06/TTT3_iKon...,TTT3,iKon936-1,PSJ0147+4630,SDSSg,16.79616,2025-11-07 01:24:05.673433,2025-11-06,2025-11-07
1047351,TTT3_iKon936-1_2025-11-07-01-26-12-046007_PSJ0...,/home/oscarsoler/work/red/2025-11-06/TTT3_iKon...,TTT3,iKon936-1,PSJ0147+4630,SDSSr,16.79616,2025-11-07 01:26:12.046007,2025-11-06,2025-11-07


In [263]:
grav_lens = ['QSO0957+561', 'Q2237+030', 'MG1654+1346', 'SDSSJ1004+4112', 'LBQS1333+0113', 'SDSSJ0819+5356',
             'EinsteinCross', 'DESI-350.3458-03.5082', 'ZTF25abnjznp']
# EinsteinCross and Q2237+030 are the same object (?)
grav_lens_ra = ['10 01 20.692 h', '22 40 30.234 h', '16 54 41.796 h', '10 04 34.936 h', '13:35:34.8 h', '08 19 59.764 h', '22 40 30.271 h', '350.3458d', '07:16:34.5h']
grav_lens_dec = ['+55 53 55.59 d', '+03 21 30.63 d', '+13 46 21.34 d', '+41 12 42.66 d', '+01 18 05.5 d', '+53 56 24.63 d', '+03 21 31.03 d', '-03.5082d', '+38:21:08d']
grav_data = []
for i in range(len(grav_lens)):
    grav_data.append({
        'name' : grav_lens[i],
        'ra' : Angle(grav_lens_ra[i]),
        'dec' : Angle(grav_lens_dec[i])
    })
df_grav_lens = pd.DataFrame(grav_data).sort_values('name').reset_index(drop=True)
df_grav_lens


,name,ra,dec
0,DESI-350.3458-03.5082,350d20m44.88s,-3d30m29.52s
1,EinsteinCross,22h40m30.271s,3d21m31.03s
2,LBQS1333+0113,13h35m34.8s,1d18m05.5s
3,MG1654+1346,16h54m41.796s,13d46m21.34s
4,Q2237+030,22h40m30.234s,3d21m30.63s
5,QSO0957+561,10h01m20.692s,55d53m55.59s
6,SDSSJ0819+5356,8h19m59.764s,53d56m24.63s
7,SDSSJ1004+4112,10h04m34.936s,41d12m42.66s
8,ZTF25abnjznp,7h16m34.5s,38d21m08s


In [303]:
df_grav_lens.index[df_grav_lens['name']=='Q227+030'].tolist() == []
df_grav_lens['name'].iloc[3]



'MG1654+1346'

In [ ]:
# grav_lens_objects = {{
#     'QSO0957+561' : {
#         'name' : 'QSO0957+561',
#         'ra' : ,
#         'dec' : 
#     }
#     'Q2237+030'
#     'MG1654+1346'
#     'SDSSJ1004+4112'
#     'LBQS1333+0113'
#     'SDSSJ0819+5356'
#     'EinsteinCross'
#     'DESI-350.3458-03.5082'
#     'ZTF25abnjznp'
# }}

# for g in grav_lens:
#     try: print(g, df.value_counts(['object']).loc[(g)])
#     except: print()


# Filter the DataFrame
df_filtered = df[df["object"].isin(grav_lens)].copy()

# Group by day and object, count observations
daily_counts = (
    df_filtered.groupby(["day", "object"])
    .size()
    .reset_index(name="count")
    )

# daily_counts.groupby(['filter']).size()
daily_counts

## Timeline overview of gravitational lens object observations

In [155]:

# Pivot for plotting
pivoted = daily_counts.pivot(index="day", columns="object", values="count").fillna(0)
pivot_for_plot = pivoted.where(pivoted != 0, np.nan)
# Remove days where all objects are NaN (no observations at all)
pivot_for_plot = pivot_for_plot.dropna(how="all")
# optional: sort index (dates) to ensure proper time order
pivot_for_plot = pivot_for_plot.sort_index()
# Plot
fig, ax = plt.subplots()

pivot_for_plot.plot(kind="line", marker="o", lw=0, alpha=0.5, figsize=(10, 5), ax=ax)


ax.set_title("Daily Observations by Object")
ax.minorticks_on()
ax.set_xlabel("Date")
ax.tick_params('x', rotation = 45)
ax.set_ylabel("Number of Observations")
ax.legend(title="Object")
ax.grid(alpha=0.5)
plt.tight_layout()
plt.show()

In [156]:
plt.savefig('observation_history_2025-11-09.png', dpi=200)

(0.12156862745098039, 0.4666666666666667, 0.7058823529411765)

## Overview of timeline observations separated by filter for each object

In [157]:
# Assiging each filter a color
filters_list = df_filtered.groupby('filter').size().keys().to_list()
n_filters = len(filters_list)

colors_dict = {filters_list[i]: plt.cm.tab10.colors[i] for i in range(n_filters)}
colors_list = [plt.cm.tab10.colors[i] for i in range(n_filters)]

# Create figure
# CORRECT to adjust to total number of filters
fig, ax = plt.subplots(4, 2)

for i, obj in enumerate(grav_lens):
    # Group by date, object, filter and count observations
    summary = (
        df_filtered[df_filtered['object']==obj].groupby(['day', 'object', 'filter'])
        .size()
        .reset_index(name='n_observations')
    )
    # Pivot the grouped data: rows are dates, columns are (object, filter) multiindex, values are counts
    pivoted = summary.pivot_table(
                                index='day',
                                columns=['object', 'filter'],
                                values='n_observations',
                                fill_value=0
)
# Plot as stacked bar chart
    #piv_plot = pivoted[pivoted['object']==obj].copy()
    iax = i//2
    jax = i%2
    # extract filters used for the object
    filters_object = summary.groupby('filter').size().keys().to_list()
    colors_object = [colors_dict.get(f, '#333333') for f in filters_object]
    pivoted.plot(
        kind='bar',
        stacked=True,
        figsize=(15, 7),
        ax=ax[iax, jax],
        legend = False,
        color = colors_object
    )
    ax[iax, jax].locator_params(axis='x', nbins=5)
    ax[iax, jax].tick_params('x', rotation = 0)
    ax[iax, jax].set_xlabel('')
    ax[iax, jax].set_title(obj)

    if iax == ax.shape[0]-1:
        ax[iax, jax].set_xlabel('Observation Date')
    if jax==0:
       ax[iax, jax].set_ylabel('Number of\nObservations')
    else:
        ax[iax, jax].set_ylabel('')

fig.suptitle('Daily Number of Observations per Object and Filter')
# Custom legend handles
ax[-1,-1].set_axis_off()
handles = [patches.Patch(color = colors_list[i], label =filters_list[i]) for i in range(n_filters)]
ax[-1,-1].legend(title='Filter', #bbox_to_anchor=(1.05, 1), loc='lower center',
           handles = handles, ncols = n_filters)
plt.tight_layout()
plt.show()

IndexError: index 4 is out of bounds for axis 0 with size 4

In [92]:
plt.savefig('grav_lens_overview_object_date_filter.png', dpi=200)

In [96]:
filters_list

['SDSSg', 'SDSSi', 'SDSSr', 'SDSSu', 'SDSSy', 'SDSSzs']

In [159]:
print(df_filtered.groupby(['object', 'filter', 'telescope']).size())

object                 filter  telescope
DESI-350.3458-03.5082  SDSSg   TTT3          2
                       SDSSi   TTT3          2
                       SDSSr   TTT3          2
                       SDSSzs  TTT3          2
EinsteinCross          SDSSg   TTT3         36
                                            ..
SDSSJ1004+4112         SDSSr   TTT2         28
ZTF25abnjznp           SDSSg   TTT3          5
                       SDSSi   TTT3          5
                       SDSSr   TTT3          5
                       SDSSzs  TTT3          5
Length: 61, dtype: int64


# Future work

In [ ]:
# ------- FUTURE WORK: IMAGE DATA HISTOGRAM ---------
# data = iv.read_data(1)
print(np.mean(data))
print(np.median(data))
print(np.std(data))
print(np.max(data),'-', np.min(data))
plt.hist(data.ravel(), bins=100)
plt.yscale('log')
plt.xscale('log')
plt.show()